# Image Analysis with Amazon Bedrock Data Automation: Seeing Beyond Text

This notebook expands your journey with Amazon Bedrock Data Automation (BDA) by moving beyond text-based content to explore the rich world of visual data. While document processing lays the foundation for structured extraction, image analysis represents a fundamental shift in how we interact with information that cannot be captured in text alone.

As you work through this notebook, you'll build capabilities to:
- Extract textual information embedded within images
- Identify and analyze visual objects and logos
- Generate comprehensive descriptions of visual content
- Classify images according to standard taxonomies
- Extract custom attributes from specialized image types

This notebook demonstrates how to use Amazon Bedrock Data Automation (BDA) to analyze images and extract valuable insights. BDA can identify text, logos, objects, and generate summaries from images, making it powerful for various image analysis tasks.

In this enhanced notebook, we'll focus on the core BDA workflow for image analysis:

1. Preparing a sample image
2. Creating a BDA project with standard and custom output configurations
3. Processing the image with BDA
4. Analyzing the results

The setup cell below contains helper functions and initialization code. **It's collapsed by default** - you can expand it to see the details, but you don't need to understand all of it to follow the main BDA workflow.

In [ ]:
# This cell contains setup code and helper functions.
# You can expand it to see the details, but you don't need to understand all of it.

import boto3
import json
import uuid
import time
import os
import sagemaker
from datetime import datetime
from IPython.display import Image, clear_output, display, Markdown, HTML
from PIL import Image as PILImage, ImageDraw
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import all utilities from the consolidated utils
from utils.utils import BDAImageUtils, show_business_context, download_image, ensure_bda_results_dir, visualize_detections

# Initialize our utility class
bda_utils = BDAImageUtils()

# Display comprehensive business context for image analysis
show_business_context("image_analysis_complete")

print(f"Setup complete. BDA utilities initialized for region: {bda_utils.current_region}")
print(f"Using S3 bucket: {bda_utils.bucket_name}")

## Industry Applications

As you work through this notebook, consider how image analysis could transform workflows in your specific domain:

**Retail**: How would automatic product detection, logo identification, and attribute extraction from product images change your catalog management and customer experiences?

**Marketing & Advertising**: What if your campaigns could automatically measure brand presence, analyze ad composition, and extract text from visual promotional materials?

**Manufacturing**: How could visual quality inspection, defect detection, and component identification from product images transform your production processes?

**Healthcare**: What new possibilities would emerge from analyzing medical images, identifying visual indicators, and extracting measurements from diagnostic visuals?

**Your Industry**: What types of images are critical to your organization's workflows? What insights within these images would provide the most business value if automatically extracted?

## 1. Prepare Sample Image

First, we'll download a sample advertisement image and upload it to S3 for processing with BDA. This image contains text, logos, and other elements that BDA can analyze.

The image will be stored in an S3 bucket that BDA can access. This is a required step as BDA needs to read the image from S3.

In [ ]:
# Download sample image
sample_image = 'travel.png'
source_url = f'https://d1xvhy22zmw77y.cloudfront.net/tmp/{sample_image}'

# Download the image locally using enhanced download function
local_path = download_image(source_url, sample_image) or bda_utils.download_image(source_url, sample_image)

# Display the image
display(Image(sample_image, width=800))

# Upload to S3
s3_key = f'{bda_utils.data_prefix}/{sample_image}'
s3_uri = bda_utils.upload_to_s3(sample_image, s3_key)
print(f"Uploaded image to S3: {s3_uri}")

## 2. Define BDA Configuration

Before creating a BDA project, we need to define both standard and custom output configurations. These configurations determine what information BDA will extract from the image.

### BDA Configuration Details

### Standard Output Configuration

The standard output configuration includes:

- **Text Detection**: Extract text from the image with bounding boxes
- **Logo Detection**: Identify brand logos with bounding boxes
- **Image Summary**: Generate a descriptive summary of the image
- **IAB Categories**: Classify the image into Internet Advertising Bureau categories

### Custom Output Configuration (Blueprint)

We'll also create a custom blueprint to extract specific information about the advertisement:

- **Product Type**: What product or service is being advertised
- **Product Count**: Number of products visible in the image
- **Scene Location**: Setting or location of the scene
- **Image Background**: Type of background in the image

In [ ]:
# Display business context for custom blueprints
show_business_context("custom_blueprints")

# Define the custom blueprint for image analysis
# This blueprint defines the schema for extracting specific information from advertisement images
blueprint = {
    # Top-level class and description define the overall purpose of the blueprint
    "class": "Ad scene analysis",  # Name of the blueprint class
    "description": "This blueprint is designed to extract key information from an image depicting an Ad scene.",
    
    # Definitions section contains reusable property groups that can be referenced elsewhere
    "definitions": {
        "ProductDetails": {  # Define a reusable group of properties for product information
            "properties": {
                "product_type": {  # Extract the type of product being advertised
                  "type": "string",  # Expect a string response
                  "inferenceType": "inferred",  # This will be inferred by AI, not extracted directly
                  "instruction": "What is the primary product or service being advertised, e.g., Clothing, Electronics, Food & Beverage, etc.?"
                },
                "product_count": {  # Count the number of products visible
                    "type": "number",  # Expect a numeric response
                    "inferenceType": "inferred",
                    "instruction": "Count the number of product visible in the image."
                },
                "scene_location": {  # Describe the setting of the advertisement
                  "type": "string",
                  "inferenceType": "inferred",
                  "instruction": "Describe the setting or location of the scene, such as the type of field or surrounding buildings."
                }
            }
        }
    },
    
    # Root-level properties to extract from the image
    "properties": {
        "product_details": {  # Use the ProductDetails definition from above
            "instruction": "Detailed information about the advertised product, including its category, name, and placement within the image.",
            "$ref": "#/definitions/ProductDetails"  # Reference to the definition above
        },
        "image_background": {  # Additional property at the root level
            "type": "string",
            "inferenceType": "inferred",
            "instruction": "What is the background of the ad image? For example, \u0027Solid color\u0027, \u0027Natural landscape\u0027, \u0027Indoor\u0027, \u0027Urban\u0027, \u0027Abstract\u0027"
        }
    }
}

# Generate a unique blueprint name to avoid naming conflicts
unique_id = str(uuid.uuid4())[0:6]
blueprint_name = f"bda-image-custom-blueprint-{unique_id}"

print(f"Creating blueprint with name: {blueprint_name}")

# Create the blueprint in BDA
try:
    bp_response = bda_utils.bda_client.create_blueprint(
        blueprintName=blueprint_name,
        type='IMAGE',  # Specify this is for image analysis
        blueprintStage='DEVELOPMENT',  # Use development stage for workshop
        schema=json.dumps(blueprint),  # Convert blueprint dict to JSON string
    )
    
    blueprint_arn = bp_response.get("blueprint", {}).get("blueprintArn")
    print(f"Blueprint created successfully with ARN: {blueprint_arn}")
except Exception as e:
    print(f"Error creating blueprint: {e}")
    blueprint_arn = None

## 3. Create BDA Project

Now we'll create a BDA project with our standard and custom output configurations. The project stores the configuration settings that will be used to process images.

### Project Architecture for Image Analysis

### Key Parameters for `create_data_automation_project` API:

- **projectName**: A unique name for the project
- **projectDescription**: A description of the project's purpose
- **projectStage**: The stage of the project (DEVELOPMENT or LIVE)
- **standardOutputConfiguration**: The standard output settings
- **customOutputConfiguration**: The custom blueprint configuration

The project ARN returned by this API will be used when invoking BDA to process images.

In [ ]:
# Show business context for project architecture
show_business_context("project_architecture")

# Create a BDA project with both standard and custom output configurations
print("Creating BDA project for image analysis...")
response = bda_utils.bda_client.create_data_automation_project(
    projectName=f'bda-workshop-image-project-{str(uuid.uuid4())[0:4]}',
    projectDescription='BDA workshop image sample project',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'image': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': [
                        'TEXT_DETECTION',  # Extract text from the image
                        'LOGOS'            # Identify brand logos
                    ]
                },
                'boundingBox': {
                    'state': 'ENABLED'     # Include bounding boxes for detected elements
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': [
                    'IMAGE_SUMMARY',       # Generate a summary of the image
                    'IAB'                  # Classify into IAB categories
                ]
            }
        },
    },
    customOutputConfiguration={
        'blueprints': [
            {
                'blueprintArn': blueprint_arn,
                'blueprintStage': 'DEVELOPMENT'
            },
        ]
    },
)

# Get the project ARN
image_project_arn = response.get("projectArn")
print(f"BDA project created with ARN: {image_project_arn}")

## 4. Process Image with BDA

Now we'll use the `invoke_data_automation_async` API to process our image with BDA. This API starts an asynchronous job to analyze the image and extract insights based on our project configuration.

In [ ]:
# Show business context for processing pipeline
show_business_context("processing_pipeline")

# Invoke BDA to process the image
print(f"Processing image: {s3_uri}")
print(f"Results will be stored at: s3://{bda_utils.bucket_name}/{bda_utils.output_prefix}")

# Call the invoke_data_automation_async API
response = bda_utils.bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': s3_uri  # The S3 location of our image
    },
    outputConfiguration={
        's3Uri': f's3://{bda_utils.bucket_name}/{bda_utils.output_prefix}'  # Where to store results
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': image_project_arn,  # The project we created
        'stage': 'DEVELOPMENT'                          # Must match the project stage
    },
    dataAutomationProfileArn=f'arn:aws:bedrock:{bda_utils.current_region}:{bda_utils.account_id}:data-automation-profile/us.data-automation-v1'
)

# Get the invocation ARN
invocation_arn = response['invocationArn']
print(f"Invocation ARN: {invocation_arn}")

# Wait for processing to complete
status_response = bda_utils.wait_for_completion(
    get_status_function=bda_utils.bda_runtime_client.get_data_automation_status,
    status_kwargs={'invocationArn': invocation_arn},
    completion_states=['Success'],
    error_states=['ClientError', 'ServiceError'],
    status_path_in_response='status',
    max_iterations=15,
    delay=10
)

# Check if processing was successful
if status_response['status'] == 'Success':
    output_config_uri = status_response.get("outputConfiguration", {}).get("s3Uri")
    print(f"\nImage processing completed successfully!")
    print(f"Output configuration: {output_config_uri}")
else:
    print(f"\nImage processing failed with status: {status}")
    if 'error_message' in status_response:
        print(f"Error message: {status_response['error_message']}")

## 5. Analyze BDA Results

Finally, we'll retrieve and analyze the results from BDA. We'll break this analysis into several focused sections:

1. Job metadata and overview 
2. Image summary and classification
3. Visual elements detection (text, logos)
4. Custom analysis and practical applications

### 5.1 Job Metadata and Overview

In [ ]:
# Show business context for business applications
show_business_context("business_applications")

# Load job metadata
config_data = bda_utils.read_json_from_s3(output_config_uri)

# Get standard output path
standard_output_path = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
standard_output = bda_utils.read_json_from_s3(standard_output_path)

# Get custom output path
custom_output_path = config_data["output_metadata"][0]["segment_metadata"][0]["custom_output_path"]
custom_output = bda_utils.read_json_from_s3(custom_output_path)

# Display job metadata
print("=== Job Information ===")
print(f"Job ID: {config_data.get('job_id', 'N/A')}")
print(f"Input type: {config_data.get('input_type', 'N/A')}")
print(f"Asset count: {len(config_data.get('output_metadata', []))}")

# Display image metadata
print("\n=== Image Metadata ===")
metadata = standard_output["metadata"]
print(f"Width: {metadata.get('image_width_pixels', 'N/A')} pixels")
print(f"Height: {metadata.get('image_height_pixels', 'N/A')} pixels")
print(f"Format: {metadata.get('image_format', 'N/A')}")

### 5.2 Image Summary and Classification

In [ ]:
# Show business context for image classification
show_business_context("image_classification")

# Display image summary
print("=== Image Summary ===")
print(standard_output["image"]["summary"])

# Display IAB categories
print("\n=== IAB Categories ===")
for iab in standard_output["image"]["iab_categories"]:
    # Check if confidence key exists before trying to access it
    if 'confidence' in iab:
        print(f"- {iab['category']} (Confidence: {iab['confidence']:.2f})")
    else:
        print(f"- {iab['category']}")

### 5.3 Visual Elements Detection (Text & Logos)

In [ ]:
# Show business context for text extraction
show_business_context("text_in_image")

# Display detected text
print("=== Detected Text ===")
if "text_lines" in standard_output["image"]:
    for i, txt in enumerate(standard_output["image"]["text_lines"]):
        print(f"{i+1}. {txt['text']}")
else:
    print("No text detected")

# Display detected logos
print("\n=== Detected Logos ===")
if "logos" in standard_output["image"]:
    for i, logo in enumerate(standard_output["image"]["logos"]):
        print(f"{i+1}. {logo.get('name', 'Unknown logo')}")
else:
    print("No logos detected")
    
# Use our enhanced visualization function to show detections
visualization = visualize_detections(sample_image, standard_output)
visualization.show()

### 5.4 Custom Analysis and Practical Applications

In [ ]:
# Create bda-results directory if it doesn't exist
ensure_bda_results_dir()

# Combine standard and custom outputs into a single result
combined_result = {
    "standard_output": standard_output,
    "custom_output": custom_output
}

# Save the combined result to the bda-results directory
with open('../bda-results/image_result.json', 'w') as f:
    json.dump(combined_result, f)
    
print(f"Saved image results to: ../bda-results/image_result.json")

# Display custom analysis results
print("\n=== Custom Analysis ===")
product_details = custom_output["inference_result"]["product_details"]
print(f"Product Type: {product_details['product_type']}")
print(f"Product Count: {product_details['product_count']}")
print(f"Scene Location: {product_details['scene_location']}")
print(f"Image Background: {custom_output['inference_result']['image_background']}")

## Summary

In this notebook, we demonstrated how to use Amazon Bedrock Data Automation (BDA) to analyze images and extract valuable insights. We covered the key steps in the BDA workflow and explored the powerful capabilities BDA offers for visual content analysis.

Through this hands-on exercise, we've seen how BDA can transform unstructured visual content into structured, actionable data that can power intelligent applications across various domains.

The journey from document processing to image analysis represents a significant expansion of our multimodal capabilities, laying the groundwork for even more advanced modalities in the upcoming modules.